In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-eu.apache.org/dist/spark/spark-2.3.3/spark-2.3.3-bin-hadoop2.7.tgz
!tar xf spark-2.3.3-bin-hadoop2.7.tgz
!pip install -q findspark

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.3-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

In [0]:
# Create a spark session
spark = SparkSession.builder.appName('lrex').getOrCreate()

In [0]:
from pyspark.ml.regression import LinearRegression

In [0]:
training = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

In [8]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [0]:
# Creating instance of the model
lr = LinearRegression(featuresCol='features', labelCol='label',
                      predictionCol='prediction')

In [0]:
lrModel = lr.fit(training)

In [11]:
lrModel.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [12]:
lrModel.intercept

0.14228558260358093

In [0]:
training_summary = lrModel.summary

In [14]:
training_summary.r2 #Variance explained by model

0.027839179518600154

In [15]:
training_summary.rootMeanSquaredError

10.16309157133015

In [0]:
all_data = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

In [0]:
train_data, test_data = all_data.randomSplit([0.7,0.3]) #train-test split 

In [21]:
train_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                346|
|   mean|0.21873187608174244|
| stddev| 10.034404181082776|
|    min| -23.51088409032297|
|    max| 27.111027963108548|
+-------+-------------------+



In [22]:
test_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                155|
|   mean|0.34206497287359655|
| stddev| 10.957396821209096|
|    min|-28.571478869743427|
|    max|  27.78383192005107|
+-------+-------------------+



In [0]:
correct_model = lr.fit(train_data)

In [24]:
test_results = correct_model.evaluate(test_data)
test_results.rootMeanSquaredError

11.145406219469672

In [0]:
unlabeled_data = test_data.select('features')

In [26]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [0]:
predictions = correct_model.transform(unlabeled_data)

In [28]:
predictions.show()

+--------------------+-------------------+
|            features|         prediction|
+--------------------+-------------------+
|(10,[0,1,2,3,4,5,...|  -1.66328978711128|
|(10,[0,1,2,3,4,5,...|-1.6929623045338034|
|(10,[0,1,2,3,4,5,...| 3.2244960796231568|
|(10,[0,1,2,3,4,5,...|-3.4768415107729376|
|(10,[0,1,2,3,4,5,...|0.02096927088308978|
|(10,[0,1,2,3,4,5,...| 3.6661535546144934|
|(10,[0,1,2,3,4,5,...| 1.6403687217127092|
|(10,[0,1,2,3,4,5,...| 0.9968612704911689|
|(10,[0,1,2,3,4,5,...|-2.3512481041254727|
|(10,[0,1,2,3,4,5,...| 0.6735193771737941|
|(10,[0,1,2,3,4,5,...| 2.0111940680029208|
|(10,[0,1,2,3,4,5,...|   5.64747591843449|
|(10,[0,1,2,3,4,5,...| 3.3930867360314907|
|(10,[0,1,2,3,4,5,...|0.14703616930644117|
|(10,[0,1,2,3,4,5,...|-1.0570102020001673|
|(10,[0,1,2,3,4,5,...| 0.3816772606758486|
|(10,[0,1,2,3,4,5,...|-3.7098395687398855|
|(10,[0,1,2,3,4,5,...| 2.4551445437943644|
|(10,[0,1,2,3,4,5,...| 2.1517742565242437|
|(10,[0,1,2,3,4,5,...| 1.9978111817778403|
+----------

### Working on our data

In [0]:
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [0]:
data = spark.read.csv('Ecommerce-Customers.csv',inferSchema=True,
                      header=True)

In [32]:
data.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [33]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



### Setup df for ML

In [0]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [35]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [0]:
assembler = VectorAssembler(inputCols=['Avg Session Length',
                                       'Time on App',
                                       'Time on Website',
                                       'Length of Membership'],
                            outputCol='features')

In [0]:
output = assembler.transform(data)

In [38]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [0]:
final_data = output.select('features','Yearly Amount Spent')

In [41]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [0]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [43]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                348|
|   mean| 498.06487144600027|
| stddev|  75.86218195738833|
|    min| 256.67058229005585|
|    max|  725.5848140556806|
+-------+-------------------+



In [44]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                152|
|   mean|  502.1739728032058|
| stddev|  86.90143110312151|
|    min|  282.4712457199145|
|    max|  765.5184619388373|
+-------+-------------------+



In [0]:
lr = LinearRegression(labelCol = 'Yearly Amount Spent')
lr_model = lr.fit(train_data)

In [0]:
test_results = lr_model.evaluate(test_data)

In [48]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| -5.127369403090654|
|-12.486942216466048|
| -7.514091817437475|
| -1.628623774188668|
| -4.162466113710025|
| 22.061175897732028|
| 2.9416756025092354|
|-6.0779353669287275|
| -14.55938706073016|
| -1.932216465330498|
|-5.6400462960157824|
|  6.178282929289878|
| 3.1953986836832087|
|  -3.44542599792112|
| -17.27187610085008|
| -13.76270374577848|
|  8.880961472014064|
| -18.09827782344223|
| -3.656094504574071|
| 0.5746379783337829|
+-------------------+
only showing top 20 rows



In [49]:
test_results.rootMeanSquaredError

10.23553836737399

In [50]:
test_results.r2

0.9860352178625025

In [51]:
 final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [0]:
unlabeled_data = test_data.select('features')

In [53]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[30.4925366965402...|
|[31.0662181616375...|
|[31.1280900496166...|
|[31.2606468698795...|
|[31.2681042107507...|
|[31.2834474760581...|
|[31.3091926408918...|
|[31.4474464941278...|
|[31.5741380228732...|
|[31.5761319713222...|
|[31.6253601348306...|
|[31.6548096756927...|
|[31.7366356860502...|
|[31.8124825597242...|
|[31.9048571310136...|
|[31.9365486184489...|
|[31.9549038566348...|
|[31.9563005605233...|
|[31.9673209478824...|
|[32.0047530203648...|
+--------------------+
only showing top 20 rows



In [0]:
predictions = lr_model.transform(unlabeled_data)

In [55]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.4925366965402...| 287.5986151230052|
|[31.0662181616375...| 461.4202354241404|
|[31.1280900496166...| 564.7667785644921|
|[31.2606468698795...|422.95525503114004|
|[31.2681042107507...|427.63299928753395|
|[31.2834474760581...| 569.7199135279354|
|[31.3091926408918...| 429.7790422374244|
|[31.4474464941278...|424.68067746215274|
|[31.5741380228732...|  558.968659221317|
|[31.5761319713222...| 543.1588004546588|
|[31.6253601348306...|   381.97694705294|
|[31.6548096756927...|469.08514079825864|
|[31.7366356860502...| 493.7380475718487|
|[31.8124825597242...|396.25577098171834|
|[31.9048571310136...| 491.2217335236662|
|[31.9365486184489...| 440.9620886411067|
|[31.9549038566348...| 431.1169184679129|
|[31.9563005605233...|  565.224209570641|
|[31.9673209478824...| 449.4059357442263|
|[32.0047530203648...|463.17134314229565|
+--------------------+------------